In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np, pandas as pd
import pickle
import os

if not os.path.isdir("./lut_plot"):
    os.mkdir("./lut_plot")

In [ ]:
def get_meas(path):
    measurements = pickle.load(open(path, "rb"))
    df = pd.DataFrame(measurements, columns=["ampl","mag_gen_sq","mag_feedback_sq","phase_diff"])
    df["mag_gen"] = np.sqrt(df["mag_gen_sq"])
    df["mag_feedback"] = np.sqrt(df["mag_feedback_sq"])
    return df

In [ ]:
df = get_meas("./measurements.pkl")
df_lut = get_meas("./measurements_lut.pkl")
df_lut_sq = get_meas("./measurements_lut_sq.pkl")
df_tab = get_meas("./measurements_tab.pkl")

In [ ]:
df.plot()

In [ ]:
def get_slope(ampl, mag_feedback):
    slope, intersect = np.polyfit(x = ampl[0:20], y = mag_feedback[0:20], deg = 1)
    return slope, intersect

#get_slope(df["ampl"], df["mag_feedback"])

In [ ]:
def get_fac(ampl, mag_feedback):
    slope, intersect = get_slope(ampl, mag_feedback)
    return[(x*slope + intersect) / y for (x,y) in zip(ampl, mag_feedback)]
def interp(x): return np.interp(x, df["ampl"], fac)

In [ ]:
x = np.linspace(0.1, 0.5, num = 50)
slope, intersect = get_slope(df["ampl"], df["mag_feedback"])

plt.plot(df["ampl"], df["mag_feedback"], label="measurement")
plt.plot(x, x*slope + intersect, label = "linear model")

plt.legend(loc=0)
plt.title("Original Measurement")

plt.savefig("./lut_plot/original_measurement.png")

In [ ]:
slope, intersect = get_slope(df["ampl"], df["mag_feedback"])

plt.plot(df_lut["ampl"], df_lut["mag_feedback"], label="measurement")
plt.plot(x, x*slope + intersect, label = "linear model")

plt.legend(loc=0)
plt.title("Lut Measurement")

plt.savefig("./lut_plot/lut_measurement.png")

In [ ]:
slope, intersect = get_slope(df["ampl"], df["mag_feedback"])

plt.plot(df_lut_sq["ampl"], df_lut_sq["mag_feedback"], label="measurement")
plt.plot(x, x*slope + intersect, label = "linear model")

plt.legend(loc=0)
plt.title("Lut Squared Measurement")

plt.savefig("./lut_plot/lut_sq_measurement.png")

In [ ]:
slope, intersect = get_slope(df_tab["ampl"], df_tab["mag_feedback"])

plt.plot(df_tab["ampl"], df_tab["mag_feedback"], label="measurement")
plt.plot(x, x*slope + intersect, label = "linear model")

plt.legend(loc=0)
plt.title("Lut Tab Measurement")

plt.savefig("./lut_plot/lut_tab_measurement.png")

In [ ]:
slope, intersect = get_slope(df["ampl"], df["mag_feedback"])

plt.plot(df["ampl"], df["mag_feedback"], label="measurement")
plt.plot(df_lut["ampl"], df_lut["mag_feedback"], label="measurement lut")
plt.plot(df_lut_sq["ampl"], df_lut_sq["mag_feedback"], label="measurement lut sq")
plt.plot(df_tab["ampl"], df_tab["mag_feedback"], label="measurement tab sq")
plt.plot(x, x*slope + intersect, label = "linear model")

plt.legend(loc=0)
plt.title("All Measurements")

plt.savefig("./lut_plot/all_measurement.png")

In [ ]:
fac = get_fac(df["ampl"], df["mag_feedback"])

In [ ]:
pickle.dump({"ampl":list(df["ampl"]),"fac":[f for f in fac]}, open("lut.pkl", "wb"))

In [ ]:
pickle.dump({"ampl":list(df["ampl"]),"fac":[f**2 for f in fac]}, open("lut_sq.pkl", "wb"))

In [ ]:
slope, intersect = get_slope(df["ampl"], df["mag_feedback"])
ampl_corr = []
for ampl in df["ampl"]:
    y = ampl * slope + intersect
    mag_feedback_idx = np.argmin(np.abs(df["mag_feedback"] - y))
    ampl_corr.append(df["ampl"][mag_feedback_idx])

In [ ]:
ampl_corr = np.array(ampl_corr)

In [ ]:
idx_start = np.argmax(ampl_corr != ampl_corr.min()) - 1
idx_end  = np.argmin(ampl_corr != ampl_corr.max()) 
start = 1.0 * idx_start / len(ampl_corr)
end   = 1.0 * idx_end / len(ampl_corr)

In [ ]:
orig_range = np.linspace(start,end,num=len(ampl_corr))
new_range  = np.linspace(0,1,num=len(ampl_corr))

ampl_corr_new = []
for f in new_range:
    ratio = np.interp(f, new_range, orig_range)
    new_ampl = np.interp(ratio, new_range, ampl_corr)
    ampl_corr_new.append(new_ampl)
ampl_corr_new

In [ ]:
fac = ampl_corr_new / df["ampl"]
pickle.dump({"ampl":list(df["ampl"]),"fac":[f for f in list(fac)]}, open("lut_tab.pkl", "wb"))

In [ ]:
pickle.load(open("./lut_tab.pkl"))